## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..//Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.65,115.33,62.92,71,91,14.29,overcast clouds
1,1,Hilo,US,19.73,-155.09,72.25,93,92,4.00,overcast clouds
2,2,Cape Town,ZA,-33.93,18.42,76.12,58,20,13.80,few clouds
3,3,Paamiut,GL,61.99,-49.67,13.53,73,6,6.87,clear sky
4,4,Shenjiamen,CN,29.96,122.30,63.03,65,29,8.46,scattered clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Vacation Criteria
# Input statements
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 40
What is the maximum temperature you would like for your trip? 70


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Create df that contains all cities that meet temperature criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.65,115.33,62.92,71,91,14.29,overcast clouds
4,4,Shenjiamen,CN,29.96,122.30,63.03,65,29,8.46,scattered clouds
7,7,Constitucion,CL,-35.33,-72.42,54.48,80,96,3.65,overcast clouds
13,13,Nikolskoye,RU,59.70,30.79,44.49,86,100,9.13,overcast clouds
14,14,Ancud,CL,-41.87,-73.82,49.89,81,71,1.52,broken clouds
15,15,Carnarvon,AU,-24.87,113.63,66.47,62,23,17.27,few clouds
20,20,Corinto,GR,37.94,22.96,68.90,62,22,7.54,few clouds
23,23,Torbay,CA,47.67,-52.73,59.76,93,75,25.32,broken clouds
24,24,Ushuaia,AR,-54.80,-68.30,44.26,93,0,0.00,clear sky
26,26,Rokytne,UA,51.28,27.21,53.13,75,83,10.89,broken clouds


In [7]:
len(preferred_cities_df)

240

In [8]:
# 4a. Determine if there are any empty rows.
# Look for null values.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities_df=preferred_cities_df.dropna()

In [10]:
len(clean_travel_cities_df)

238

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = " "
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,62.92,overcast clouds,-33.65,115.33,
4,Shenjiamen,CN,63.03,scattered clouds,29.96,122.30,
7,Constitucion,CL,54.48,overcast clouds,-35.33,-72.42,
13,Nikolskoye,RU,44.49,overcast clouds,59.70,30.79,
14,Ancud,CL,49.89,broken clouds,-41.87,-73.82,
15,Carnarvon,AU,66.47,few clouds,-24.87,113.63,
20,Corinto,GR,68.90,few clouds,37.94,22.96,
23,Torbay,CA,59.76,broken clouds,47.67,-52.73,
24,Ushuaia,AR,44.26,clear sky,-54.80,-68.30,
26,Rokytne,UA,53.13,broken clouds,51.28,27.21,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!=" "]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,62.92,overcast clouds,-33.65,115.33,Observatory Guest House
4,Shenjiamen,CN,63.03,scattered clouds,29.96,122.30,Jinjiang Inn Zhoushan Putuo Shenjiamen
7,Constitucion,CL,54.48,overcast clouds,-35.33,-72.42,Hotel Boutique Las Azucenas
13,Nikolskoye,RU,44.49,overcast clouds,59.70,30.79,Tourist House - Sablino
14,Ancud,CL,49.89,broken clouds,-41.87,-73.82,Hotel Arena Gruesa


In [14]:
# 8a. Create the output File (CSV)
output_data_file="./WheatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
# Assign variable for pop-up marker to be placed on each city.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Maximum Temperature Heatmap from a hotel_df with pop-up markers.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.0)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))